Pre-COVID19

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('/home/yzwang/MF/github')))         #your own path
sys.path.append(os.path.abspath(os.path.join('/home/yzwang/MF/github/Method')))
import numpy as np
import torch
from datetime import datetime
from Baseline import rolling_forward_forecast_output_batch_v2, set_seed
from utils import (root_mean_squared_error, mean_absolute_error, print_stats, plot_forecasts_multi)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

np.printoptions(precision=4, suppress=True)
np.set_printoptions(precision=4, suppress=True)
set_seed(42)

#data parameters
d1 = 20
d2 = 3
P1 = 13
P2 = 56
QP3 = 9
Q = 5

T = 188
T_val = 20
T_test = 20
T_train = T - T_val - T_test

T_M = T * d2
T_H = T * d1 * d2
print('T',T,'T_H:', T_H, 'T_M:', T_M)
print('T_train:', T_train, 'T_val:', T_val, 'T_test:', T_test)

#hyperparameters
r1_list =  list(range(1, 9))
r2_list = list(range(3, 7))
# k_list =  list(range(1, 7)) #lag
k_list = [1, 4, 8, 12]
r1_num = len(r1_list)
r2_num = len(r2_list)
k_num = len(k_list)
epochs = 150

Y = np.load(f'../data_clean/Y_std.npy')[:,:T]
X_M = np.load(f'../data_clean/X_M_std.npy')[:,:T*d2]
X_H = np.load(f'../data_clean/X_H_std.npy')[:,:T*d1*d2]

print_stats(Y, 'Y')
print_stats(X_M, 'X_M')
print_stats(X_H, 'X_H')

# --- Data Splitting ---
X_H_train = X_H[:, :(T_train + T_val) * d1 * d2]
X_M_train = X_M[:, :(T_train + T_val) * d2]
Y_train = Y[:, :(T_train + T_val)]

X_H_test = X_H[:, T_train * d1 * d2 + T_val * d1 * d2:]
X_M_test = X_M[:, T_train * d2 + T_val * d2:]
Y_test = Y[:, T_train + T_val:]

T 188 T_H: 11280 T_M: 564
T_train: 148 T_val: 20 T_test: 20
--- Y ---
Shape: (9, 188)
Stat: max=9.0831  min=-4.5942  mean=0.0039  std=0.8721
--- X_M ---
Shape: (56, 564)
Stat: max=16.9403  min=-17.1282  mean=0.0004  std=0.8662
--- X_H ---
Shape: (13, 11280)
Stat: max=23.1950  min=-25.5649  mean=0.0001  std=0.9700


Test 2

In [2]:
set_seed(42)
r1 = 5
r2 = 2
k = 1
decay = False
early_stopping = False
epochs = 200
batch_size = 64
hidden_dim1 = 64
hidden_dim = 128
stop_threshold = 1e-6
L=2
L3=2
lr = 1e-5
# scaler = GradScaler() if device.type == 'cuda' else None
scaler = None
D2 = P2 + r1*P1
D = r1*r2*P1 + r2*P2 + QP3
print(f'\n======================== Final Test with r1={r1}, r2={r2}, k={k}, D={D}, epoch{epochs}, {scaler} ========================')

X_H_train = X_H[:, :(T_train+T_val)*d1*d2] 
X_M_train = X_M[:, :(T_train+T_val)*d2] 
Y_train = Y[:, :(T_train+T_val)] 
X_H_test = X_H[:, T_train*d1*d2 + T_val*d1*d2:]     # (P1, 0.1T*d1*d2)
X_M_test = X_M[:, T_train*d2 + T_val*d2:]           # (P2, 0.1T*d2)
Y_test = Y[:, T_train + T_val:]                     # (P3, 0.1T)

forecast_test, forecast_true = rolling_forward_forecast_output_batch_v2(
    Y_train, X_M_train, X_H_train, Y_test, X_M_test, X_H_test,
    d1=d1, d2=d2, r1=r1, r2=r2, k=k, L=L, L3=L3, epochs=epochs,
    scaler=scaler, hidden_dim=hidden_dim,
    hidden_dim1 = hidden_dim1, hidden_dim2 = hidden_dim1,
    batch_size=batch_size, lr=lr,         
    device=device, print_importance=True
)

# Calculate forecast errors
err_MAE_t, err_MAE = mean_absolute_error(forecast_true, forecast_test)
err_t, err = root_mean_squared_error(forecast_true, forecast_test)
print("\n=== Results ===")
print("True matrix:", forecast_true.T)
print('forecast matrix:', np.round(forecast_test.T,4))
print("\nMAE:", err_MAE_t)
print("RMSE:", err_t)
print("Mean MAE:", err_MAE)
print("Mean RMSE:", err)
print(f'=====================r1={r1}, r2={r2}, k={k}, D={D}, epoch{epochs}====================')
now = datetime.now()
print('current time:', now.strftime("%Y-%m-%d %H:%M"))
# plot_forecasts_multi(forecast_true, forecast_test, save_path=f'../result/DSNN3_r{r1}{r2}_k{k}_L{L}{L3}_hidden{hidden_dim1}{hidden_dim}_epoch{epochs}.png')


======================== Final Test with r1=5, r2=2, k=1, D=251, epoch200, None ========================
Step 0, Epoch 0, Train Loss: 0.724759
Step 0, Epoch 90, Train Loss: 0.712458
Step 0, Epoch 180, Train Loss: 0.700863

INPUT IMPORTANCE: DNN_g1 (High-frequency → Medium factors)
d1 = 20 lags (from lag -19 to lag 0)
  Lag |     Weight-based |     Gradient-based
------------------------------------------------
  -19 |    4.51% |     5.10%
  -18 |    4.59% |     4.46%
  -17 |    4.64% |     6.02%
  -16 |    5.72% |     9.29%
  -15 |    4.97% |     6.13%
  -14 |    5.26% |     3.88%
  -13 |    5.39% |     3.69%
  -12 |    4.71% |     5.62%
  -11 |    5.29% |     7.63%
  -10 |    4.98% |     3.64%
   -9 |    4.85% |     4.51%
   -8 |    5.18% |     5.15%
   -7 |    5.40% |     6.07%
   -6 |    5.05% |     4.40%
   -5 |    5.51% |     3.59%
   -4 |    4.87% |     4.55%
   -3 |    4.68% |     3.59%
   -2 |    4.57% |     4.91%
   -1 |    4.52% |     3.43%
    0 |    5.31% |     4.33%

INPU

During & Post COVID

In [3]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('/home/yzwang/MF/github')))
sys.path.append(os.path.abspath(os.path.join('/home/yzwang/MF/github/Method')))
import numpy as np
import torch
from datetime import datetime
from Baseline import rolling_forward_forecast_output_batch_v1, set_seed
from utils import (root_mean_squared_error, mean_absolute_error, print_stats, plot_forecasts_multi)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

np.printoptions(precision=4, suppress=True)
np.set_printoptions(precision=4, suppress=True)
set_seed(42)

#data parameters
d1 = 20
d2 = 3
P1 = 13
P2 = 56
QP3 = 9
Q = 5

T = 208
T_val = 20
T_test = 20
T_train = T - T_val - T_test

T_M = T * d2
T_H = T * d1 * d2
print('T',T,'T_H:', T_H, 'T_M:', T_M)
print('T_train:', T_train, 'T_val:', T_val, 'T_test:', T_test)

#hyperparameters
r1_list =  list(range(1, 9))
r2_list = list(range(3, 7))
# k_list =  list(range(1, 7)) #lag
k_list = [1, 4, 8, 12]
r1_num = len(r1_list)
r2_num = len(r2_list)
k_num = len(k_list)
epochs = 150

Y = np.load(f'../data_clean/Y_std.npy')[:,:T]
X_M = np.load(f'../data_clean/X_M_std.npy')[:,:T*d2]
X_H = np.load(f'../data_clean/X_H_std.npy')[:,:T*d1*d2]

print_stats(Y, 'Y')
print_stats(X_M, 'X_M')
print_stats(X_H, 'X_H')

# --- Data Splitting ---
X_H_train = X_H[:, :(T_train + T_val) * d1 * d2]
X_M_train = X_M[:, :(T_train + T_val) * d2]
Y_train = Y[:, :(T_train + T_val)]

X_H_test = X_H[:, T_train * d1 * d2 + T_val * d1 * d2:]
X_M_test = X_M[:, T_train * d2 + T_val * d2:]
Y_test = Y[:, T_train + T_val:]

T 208 T_H: 12480 T_M: 624
T_train: 168 T_val: 20 T_test: 20
--- Y ---
Shape: (9, 208)
Stat: max=9.0831  min=-8.9814  mean=-0.0000  std=1.0000
--- X_M ---
Shape: (56, 624)
Stat: max=24.6946  min=-24.2957  mean=0.0000  std=1.0000
--- X_H ---
Shape: (13, 12480)
Stat: max=30.8164  min=-33.0392  mean=-0.0000  std=1.0000


In [4]:
r1 = 5
r2 = 4
k = 1
L = 2
L3 = 2
epochs = 200
batch_size = 64
hidden_dim1 = 64
hidden_dim = 128
lr = 1e-4
# scaler = GradScaler() if device.type == 'cuda' else None
scaler = None
D2 = P2 + r1*P1
D = r1*r2*P1 + r2*P2 + QP3
print(f'\n======================== Final Test with r1={r1}, r2={r2}, k={k}, D={D}, epoch{epochs}, {scaler} ========================')

X_H_train = X_H[:, :(T_train+T_val)*d1*d2] 
X_M_train = X_M[:, :(T_train+T_val)*d2] 
Y_train = Y[:, :(T_train+T_val)] 
X_H_test = X_H[:, T_train*d1*d2 + T_val*d1*d2:]     # (P1, 0.1T*d1*d2)
X_M_test = X_M[:, T_train*d2 + T_val*d2:]           # (P2, 0.1T*d2)
Y_test = Y[:, T_train + T_val:]                     # (P3, 0.1T)

forecast_test, forecast_true = rolling_forward_forecast_output_batch_v1(
    Y_train, X_M_train, X_H_train, Y_test, X_M_test, X_H_test,
    d1=d1, d2=d2, r1=r1, r2=r2, L=L, L3=L3, k=k, epochs=epochs,
    scaler=scaler, hidden_dim=hidden_dim,
    hidden_dim1 = hidden_dim1, hidden_dim2 = hidden_dim1,
    batch_size=batch_size, lr=lr,         
    device=device, print_importance=True
)

# Calculate forecast errors
err_MAE_t, err_MAE = mean_absolute_error(forecast_true, forecast_test)
err_t, err = root_mean_squared_error(forecast_true, forecast_test)
print("\n=== Results ===")
print("True matrix:", forecast_true.T)
print('forecast matrix:', np.round(forecast_test.T,4))
print("\nMAE:", err_MAE_t)
print("RMSE:", err_t)
print("Mean MAE:", err_MAE)
print("Mean RMSE:", err)
print(f'=====================r1={r1}, r2={r2}, k={k}, D={D}, epoch{epochs}====================')
now = datetime.now()
print('current time:', now.strftime("%Y-%m-%d %H:%M"))
# plot_forecasts_multi(forecast_true, forecast_test, save_path=f'../result/DSNN3_r{r1}{r2}_k{k}_L{L}{L3}_hidden{hidden_dim1}{hidden_dim}_epoch{epochs}.png')


======================== Final Test with r1=5, r2=4, k=1, D=493, epoch200, None ========================
Step 0, Epoch 0, Train Loss: 0.686797

INPUT IMPORTANCE: DNN_g1 (High-frequency → Medium factors)
d1 = 20 lags (from lag -19 to lag 0)
  Lag |     Weight-based |     Gradient-based
------------------------------------------------
  -19 |    5.28% |     4.55%
  -18 |    4.73% |     3.90%
  -17 |    4.81% |     5.02%
  -16 |    5.61% |     6.28%
  -15 |    4.69% |     4.19%
  -14 |    4.46% |     3.80%
  -13 |    5.04% |     5.06%
  -12 |    5.72% |     7.48%
  -11 |    5.23% |     6.05%
  -10 |    5.16% |     4.62%
   -9 |    5.49% |     6.02%
   -8 |    5.14% |     5.53%
   -7 |    4.28% |     4.25%
   -6 |    4.82% |     4.18%
   -5 |    4.80% |     4.59%
   -4 |    4.57% |     4.74%
   -3 |    5.30% |     5.10%
   -2 |    5.42% |     4.32%
   -1 |    5.05% |     5.31%
    0 |    4.41% |     5.02%

INPUT IMPORTANCE: DNN_g2 (Medium-frequency → Low factors)
d2 = 3 lags (from lag -2 